## Setup

In [1]:
%run setup.ipynb

In [2]:
region_vgsc = '2L', 2358158, 2431617
phase2_ar1.load_geneset_agamp44(attributes=['ID', 'Parent'])
geneset_agamp44 = geneset_to_pandas(phase2_ar1.geneset_agamp44)
geneset_agamp44_vgsc = geneset_agamp44.query("(seqid == %r) & (start >= %s) & (end <= %s)" % region_vgsc).copy()
geneset_davies = geneset_to_pandas(allel.FeatureTable.from_gff3('davies_vgsc_model.gff3', attributes=['ID', 'Parent']))
geneset_vgsc_combined = pandas.concat([geneset_agamp44_vgsc, geneset_davies])

In [3]:
geneset_vgsc_combined.query("type == 'mRNA'")

,seqid,source,type,start,end,score,strand,phase,ID,Parent
672,2L,VectorBase,mRNA,2358158,2431617,-1,+,-1,AGAP004707-RA,AGAP004707
673,2L,VectorBase,mRNA,2358158,2431617,-1,+,-1,AGAP004707-RB,AGAP004707
674,2L,VectorBase,mRNA,2358158,2431617,-1,+,-1,AGAP004707-RC,AGAP004707
0,2L,Davies et al. (2007),mRNA,2358158,2431617,-1,+,-1,Davies-C1N2,AGAP004707
1,2L,Davies et al. (2007),mRNA,2358158,2431617,-1,+,-1,Davies-C3N2,AGAP004707
2,2L,Davies et al. (2007),mRNA,2358158,2431617,-1,+,-1,Davies-C5N2,AGAP004707
3,2L,Davies et al. (2007),mRNA,2358158,2431617,-1,+,-1,Davies-C7N2,AGAP004707
4,2L,Davies et al. (2007),mRNA,2358158,2431617,-1,+,-1,Davies-C8N2,AGAP004707
5,2L,Davies et al. (2007),mRNA,2358158,2431617,-1,+,-1,Davies-C10N2,AGAP004707
6,2L,Davies et al. (2007),mRNA,2358158,2431617,-1,+,-1,Davies-C11N2,AGAP004707


In [4]:
def tabulate_variants(callset, chrom, start, stop):
    variants = callset[chrom]['variants']
    pos = allel.SortedIndex(variants['POS'])
    loc = pos.locate_range(start, stop)
    cols = [
        variants['CHROM'][loc],
        variants['POS'][loc],
        variants['num_alleles'][loc],
        variants['FILTER_PASS'][loc],
        variants['REF'][loc],
        variants['ALT'][loc],
        variants['AC'][loc],
        variants['ANN'][loc]['Annotation'],
        variants['ANN'][loc]['HGVS_p'],
    ]

    def split_alleles(row):
        for i in range(row.num_alleles - 1):
            yield row.chrom, row.pos, row.num_alleles, row.ref, row.alt[i], i, row.ac[i], row.filter_pass, (row.annotation if i == 0 else None), (row.hgvs_p if i == 0 else None)

        
    tbl = (
        etl
        .fromcolumns(cols, header=['chrom', 'pos', 'num_alleles', 'filter_pass', 'ref', 'alt', 'ac', 'annotation', 'hgvs_p'])
        .rowmapmany(split_alleles, header=['chrom', 'pos', 'num_alleles', 'ref', 'alt', 'alt_idx', 'alt_ac', 'filter_pass', 'annotation', 'hgvs_p'])
        .convert('chrom ref alt annotation hgvs_p'.split(), lambda v: str(v, 'ascii'))
    )
    
    return tbl


In [5]:
tbl_variants_pass = tabulate_variants(phase1_ar31.callset_pass, *region_vgsc)
tbl_variants_pass

0|chrom,1|pos,2|num_alleles,3|ref,4|alt,5|alt_idx,6|alt_ac,7|filter_pass,8|annotation,9|hgvs_p
2L,2358254,2,G,A,0,1,True,missense_variant,p.Asp33Asn
2L,2358316,2,T,G,0,73,True,intron_variant,.
2L,2358328,2,T,C,0,2,True,intron_variant,.
2L,2358353,2,C,T,0,1,True,intron_variant,.
2L,2358405,2,T,A,0,1,True,intron_variant,.


In [6]:
import veff

In [7]:
combined_genome = veff.Genome(fasta_path=phase2_ar1.genome_agamp3_fn,
                              gff3_path=[phase2_ar1.geneset_agamp44_fn, 'davies_vgsc_model.gff3'],
                              seqid='2L')
combined_genome

In [8]:
for f in combined_genome.get_children('AGAP004707'):
    print(f.feature_id)

AGAP004707-RA
AGAP004707-RB
AGAP004707-RC
Davies-C1N2
Davies-C3N2
Davies-C5N2
Davies-C7N2
Davies-C8N2
Davies-C10N2
Davies-C11N2
Davies-C1N9
Davies-C8N9
Davies-C1N9ck


In [1]:
tbl_davies_exons = (
    etl
    .fromdataframe(geneset_davies)
    .eq('type', 'CDS')
    .cutout('Parent', 'source', 'type', 'score', 'strand', 'phase')
    .merge(key=('start', 'end'))
    .rename('ID', 'exon')
    .movefield('seqid', 0)
)
tbl_davies_exons.displayall()

NameError: name 'etl' is not defined

In [10]:
def lpop(l, default=None):
    """Pop the first item from a list if not empty."""
    try:
        return l[0]
    except IndexError:
        return default


## Coding variation, PASS SNPs

In [11]:
selected_effects = [
    'NON_SYNONYMOUS_CODING', 
    'SYNONYMOUS_CODING'
]

tbl_variants_pass_eff = (
    tbl_variants_pass
    .intervalleftjoin(
        # don't include short exon variations otherwise screws up join
        tbl_davies_exons.select('exon', lambda v: v[-1] != '-'),
        lkey='chrom', rkey='chrom', lstart='pos', rstart='start', lstop='pos', rstop='end', include_stop=True)
    .cutout(10)
    .rename({10: 'exon_start', 11: 'exon_end'})
    .addfield('eff', lambda row: [e for e in veff.get_effects(combined_genome, chrom=row.chrom, pos=row.pos, ref=row.ref, alt=row.alt) if e.effect in selected_effects])
    .addfield('RA', lambda row: lpop([e.aa_change for e in row.eff if e.transcript_id == 'AGAP004707-RA'], ''))
    .addfield('RB', lambda row: lpop([e.aa_change for e in row.eff if e.transcript_id == 'AGAP004707-RB'], ''))
    .addfield('RC', lambda row: lpop([e.aa_change for e in row.eff if e.transcript_id == 'AGAP004707-RC'], ''))
    .addfield('C1N2', lambda row: lpop([e.aa_change for e in row.eff if e.transcript_id == 'Davies-C1N2'], ''))
    .addfield('C3N2', lambda row: lpop([e.aa_change for e in row.eff if e.transcript_id == 'Davies-C3N2'], ''))
    .addfield('C5N2', lambda row: lpop([e.aa_change for e in row.eff if e.transcript_id == 'Davies-C5N2'], ''))
    .addfield('C7N2', lambda row: lpop([e.aa_change for e in row.eff if e.transcript_id == 'Davies-C7N2'], ''))
    .addfield('C8N2', lambda row: lpop([e.aa_change for e in row.eff if e.transcript_id == 'Davies-C8N2'], ''))
    .addfield('C10N2', lambda row: lpop([e.aa_change for e in row.eff if e.transcript_id == 'Davies-C10N2'], ''))
    .addfield('C11N2', lambda row: lpop([e.aa_change for e in row.eff if e.transcript_id == 'Davies-C11N2'], ''))
    .addfield('C1N9', lambda row: lpop([e.aa_change for e in row.eff if e.transcript_id == 'Davies-C1N9'], ''))
    .addfield('C8N9', lambda row: lpop([e.aa_change for e in row.eff if e.transcript_id == 'Davies-C8N9'], ''))
    .addfield('C1N9ck', lambda row: lpop([e.aa_change for e in row.eff if e.transcript_id == 'Davies-C1N9ck'], ''))
    .cutout('eff', 'filter_pass')
    .select(lambda row: row.exon)
    .cache()
)

In [12]:
def veff_aa_style(v):
    return 'background-color: #%s' % ('fff' if v in {None, '.', ''} else 'ddf' if v[0] == v[-1] else 'faa')
    

tbl_variants_pass_eff.displayall(
    td_styles={
        'num_alleles': lambda v: 'background-color: #%s' % ('ff0' if v > 2 else 'fff'),
        'alt_ac': lambda v: 'background-color:rgba(0, 255, 0, %.3f)' % (min(1, v/100)),
        'hgvs_p': lambda v: 'background-color: #%s' % ('fff' if v in {None, '.'} else 'ddf' if v[2:5] == v[-3:] else 'faa'),
        'RA': veff_aa_style,
        'RB': veff_aa_style,
        'RC': veff_aa_style,
        'C1N2': veff_aa_style,
        'C3N2': veff_aa_style,
        'C5N2': veff_aa_style,
        'C7N2': veff_aa_style,
        'C8N2': veff_aa_style,
        'C10N2': veff_aa_style,
        'C11N2': veff_aa_style,
        'C1N9': veff_aa_style,
        'C8N9': veff_aa_style,
        'C1N9ck': veff_aa_style,
    }
)

0|chrom,1|pos,2|num_alleles,3|ref,4|alt,5|alt_idx,6|alt_ac,7|annotation,8|hgvs_p,9|exon_start,10|exon_end,11|exon,12|RA,13|RB,14|RC,15|C1N2,16|C3N2,17|C5N2,18|C7N2,19|C8N2,20|C10N2,21|C11N2,22|C1N9,23|C8N9,24|C1N9ck
2L,2358254,2,G,A,0,1,missense_variant,p.Asp33Asn,2358158,2358304,1,D33N,D33N,D33N,D33N,D33N,D33N,D33N,D33N,D33N,D33N,D33N,D33N,D33N
2L,2361994,2,C,G,0,2,intron_variant,.,2361989,2362144,3,,,,R51R,R51R,R51R,R51R,R62R,R51R,R51R,R51R,R62R,R51R
2L,2362002,2,A,T,0,2,splice_region_variant&intron_varia,.,2361989,2362144,3,,,,D54V,D54V,D54V,D54V,D65V,D54V,D54V,D54V,D65V,D54V
2L,2362003,2,C,T,0,2,splice_region_variant&intron_varia,.,2361989,2362144,3,,,,D54D,D54D,D54D,D54D,D65D,D54D,D54D,D54D,D65D,D54D
2L,2362019,2,G,T,0,2,missense_variant,p.Gly54Cys,2361989,2362144,3,G54C,G54C,G54C,G60C,G60C,G60C,G60C,G71C,G60C,G60C,G60C,G71C,G60C
2L,2362023,2,C,T,0,1,missense_variant,p.Pro55Leu,2361989,2362144,3,P55L,P55L,P55L,P61L,P61L,P61L,P61L,P72L,P61L,P61L,P61L,P72L,P61L
2L,2381154,2,C,T,0,2,synonymous_variant,p.Arg125Arg,2381065,2381270,4,R125R,R125R,R125R,R131R,R131R,R131R,R131R,R142R,R131R,R131R,R131R,R142R,R131R
2L,2385757,2,A,T,0,1,synonymous_variant,p.Arg228Arg,2385694,2385785,6,R228R,R228R,R228R,R234R,R191R,R234R,R234R,R245R,R234R,R234R,R234R,R245R,R234R
2L,2385769,2,G,T,0,1,synonymous_variant,p.Thr232Thr,2385694,2385785,6,T232T,T232T,T232T,T238T,T195T,T238T,T238T,T249T,T238T,T238T,T238T,T249T,T238T
2L,2390130,2,C,T,0,40,splice_region_variant&synonymous_v,p.Gly238Gly,2390129,2390341,7,G238G,G238G,G238G,G244G,G201G,G244G,G244G,G255G,G244G,G244G,G244G,G255G,G244G


## Coding variation, all SNPs (i.e., including non-PASS) 

In [13]:
tbl_variants_all = tabulate_variants(phase1_ar31.callset, *region_vgsc)
tbl_variants_all

0|chrom,1|pos,2|num_alleles,3|ref,4|alt,5|alt_idx,6|alt_ac,7|filter_pass,8|annotation,9|hgvs_p
2L,2358254,2,G,A,0,1,True,missense_variant,p.Asp33Asn
2L,2358316,2,T,G,0,73,True,intron_variant,.
2L,2358328,2,T,C,0,2,True,intron_variant,.
2L,2358353,2,C,T,0,1,True,intron_variant,.
2L,2358405,2,T,A,0,1,True,intron_variant,.


In [14]:
selected_effects = [
    'NON_SYNONYMOUS_CODING', 
    'SYNONYMOUS_CODING'
]


tbl_variants_all_eff = (
    tbl_variants_all
    .intervalleftjoin(
        # don't include short exon variations otherwise screws up join
        tbl_davies_exons.select('exon', lambda v: v[-1] != '-'),
        lkey='chrom', rkey='chrom', lstart='pos', rstart='start', lstop='pos', rstop='end', include_stop=True)
    .cutout(10)
    .rename({10: 'exon_start', 11: 'exon_end'})
    .addfield('eff', lambda row: [e for e in veff.get_effects(combined_genome, chrom=row.chrom, pos=row.pos, ref=row.ref, alt=row.alt) if e.effect in selected_effects])
    .addfield('RA', lambda row: lpop([e.aa_change for e in row.eff if e.transcript_id == 'AGAP004707-RA'], ''))
    .addfield('RB', lambda row: lpop([e.aa_change for e in row.eff if e.transcript_id == 'AGAP004707-RB'], ''))
    .addfield('RC', lambda row: lpop([e.aa_change for e in row.eff if e.transcript_id == 'AGAP004707-RC'], ''))
    .addfield('C1N2', lambda row: lpop([e.aa_change for e in row.eff if e.transcript_id == 'Davies-C1N2'], ''))
    .addfield('C3N2', lambda row: lpop([e.aa_change for e in row.eff if e.transcript_id == 'Davies-C3N2'], ''))
    .addfield('C5N2', lambda row: lpop([e.aa_change for e in row.eff if e.transcript_id == 'Davies-C5N2'], ''))
    .addfield('C7N2', lambda row: lpop([e.aa_change for e in row.eff if e.transcript_id == 'Davies-C7N2'], ''))
    .addfield('C8N2', lambda row: lpop([e.aa_change for e in row.eff if e.transcript_id == 'Davies-C8N2'], ''))
    .addfield('C10N2', lambda row: lpop([e.aa_change for e in row.eff if e.transcript_id == 'Davies-C10N2'], ''))
    .addfield('C11N2', lambda row: lpop([e.aa_change for e in row.eff if e.transcript_id == 'Davies-C11N2'], ''))
    .addfield('C1N9', lambda row: lpop([e.aa_change for e in row.eff if e.transcript_id == 'Davies-C1N9'], ''))
    .addfield('C8N9', lambda row: lpop([e.aa_change for e in row.eff if e.transcript_id == 'Davies-C8N9'], ''))
    .addfield('C1N9ck', lambda row: lpop([e.aa_change for e in row.eff if e.transcript_id == 'Davies-C1N9ck'], ''))
    .cutout('eff')
    .select(lambda row: row.exon)
    .cache()
)

In [15]:
tbl_variants_all_eff.displayall(
    td_styles={
        'num_alleles': lambda v: 'background-color: #%s' % ('ff0' if v > 2 else 'fff'),
        'alt_ac': lambda v: 'background-color:rgba(0, 255, 0, %.3f)' % (min(1, v/100)),
        'filter_pass': lambda v: 'background-color: %s' % ('white' if v else 'red'),
        'hgvs_p': lambda v: 'background-color: #%s' % ('fff' if v in {None, '.'} else 'ddf' if v[2:5] == v[-3:] else 'faa'),
        'RA': veff_aa_style,
        'RB': veff_aa_style,
        'RC': veff_aa_style,
        'C1N2': veff_aa_style,
        'C3N2': veff_aa_style,
        'C5N2': veff_aa_style,
        'C7N2': veff_aa_style,
        'C8N2': veff_aa_style,
        'C10N2': veff_aa_style,
        'C11N2': veff_aa_style,
        'C1N9': veff_aa_style,
        'C8N9': veff_aa_style,
        'C1N9ck': veff_aa_style,
    }
)

0|chrom,1|pos,2|num_alleles,3|ref,4|alt,5|alt_idx,6|alt_ac,7|filter_pass,8|annotation,9|hgvs_p,10|exon_start,11|exon_end,12|exon,13|RA,14|RB,15|RC,16|C1N2,17|C3N2,18|C5N2,19|C7N2,20|C8N2,21|C10N2,22|C11N2,23|C1N9,24|C8N9,25|C1N9ck
2L,2358254,2,G,A,0,1,True,missense_variant,p.Asp33Asn,2358158,2358304,1,D33N,D33N,D33N,D33N,D33N,D33N,D33N,D33N,D33N,D33N,D33N,D33N,D33N
2L,2359670,2,G,A,0,7,False,intron_variant,.,2359640,2359672,2j,,,,,,,,E60K,,,,E60K,
2L,2361994,2,C,G,0,2,True,intron_variant,.,2361989,2362144,3,,,,R51R,R51R,R51R,R51R,R62R,R51R,R51R,R51R,R62R,R51R
2L,2362002,2,A,T,0,2,True,splice_region_variant&intron_varia,.,2361989,2362144,3,,,,D54V,D54V,D54V,D54V,D65V,D54V,D54V,D54V,D65V,D54V
2L,2362003,2,C,T,0,2,True,splice_region_variant&intron_varia,.,2361989,2362144,3,,,,D54D,D54D,D54D,D54D,D65D,D54D,D54D,D54D,D65D,D54D
2L,2362019,2,G,T,0,2,True,missense_variant,p.Gly54Cys,2361989,2362144,3,G54C,G54C,G54C,G60C,G60C,G60C,G60C,G71C,G60C,G60C,G60C,G71C,G60C
2L,2362023,2,C,T,0,1,True,missense_variant,p.Pro55Leu,2361989,2362144,3,P55L,P55L,P55L,P61L,P61L,P61L,P61L,P72L,P61L,P61L,P61L,P72L,P61L
2L,2381133,2,T,G,0,2,False,synonymous_variant,p.Leu118Leu,2381065,2381270,4,L118L,L118L,L118L,L124L,L124L,L124L,L124L,L135L,L124L,L124L,L124L,L135L,L124L
2L,2381154,2,C,T,0,2,True,synonymous_variant,p.Arg125Arg,2381065,2381270,4,R125R,R125R,R125R,R131R,R131R,R131R,R131R,R142R,R131R,R131R,R131R,R142R,R131R
2L,2385757,2,A,T,0,1,True,synonymous_variant,p.Arg228Arg,2385694,2385785,6,R228R,R228R,R228R,R234R,R191R,R234R,R234R,R245R,R234R,R234R,R234R,R245R,R234R


## Splice site variation

In [16]:
selected_effects = [
    'INTRONIC', 
    'SPLICE_CORE',
    'SPLICE_REGION',
]


def transcript_effect(transcript_id):
    def f(row):
        return lpop([(e.effect, e.intron_cds_5prime, e.intron_5prime_dist, e.intron_cds_3prime, e.intron_3prime_dist) 
                     for e in row.eff if e.transcript_id == transcript_id], '')
    return f


tbl_variants_all_eff_intronic = (
    tbl_variants_all
    .addfield('eff', lambda row: [e for e in veff.get_effects(combined_genome, chrom=row.chrom, pos=row.pos, ref=row.ref, alt=row.alt) if e.effect in selected_effects])
    .addfield('RA', transcript_effect('AGAP004707-RB'))
    .addfield('RB', transcript_effect('AGAP004707-RB'))
    .addfield('RC', transcript_effect('AGAP004707-RC'))
    .addfield('C1N2', transcript_effect('Davies-C1N2'))
    .addfield('C3N2', transcript_effect('Davies-C3N2'))
    .addfield('C5N2', transcript_effect('Davies-C5N2'))
    .addfield('C7N2', transcript_effect('Davies-C7N2'))
    .addfield('C8N2', transcript_effect('Davies-C8N2'))
    .addfield('C10N2', transcript_effect('Davies-C10N2'))
    .addfield('C11N2', transcript_effect('Davies-C11N2'))
    .addfield('C1N9', transcript_effect('Davies-C1N9'))
    .addfield('C8N9', transcript_effect('Davies-C8N9'))
    .addfield('C1N9ck', transcript_effect('Davies-C1N9ck'))
    .cutout('eff')
    .cache()
)

In [22]:
def veff_intron_style(v):
    return 'background-color: #%s' % ('fff' if v in {None, '.', ''} else 'ddf' if v[0] == 'INTRONIC' else 'fda' if v[0] == 'SPLICE_REGION' else 'faa')


(tbl_variants_all_eff_intronic
 .select(lambda row: (any([(row[k] and row[k][0].startswith('SPLICE')) for k in 'RA RB RC C1N2 C3N2 C5N2 C7N2 C8N2 C10N2 C11N2 C1N9 C8N9 C1N9ck'.split()]) or (row.annotation and 'splice' in row.annotation)))
 .displayall(
    td_styles={
        'num_alleles': lambda v: 'background-color: #%s' % ('ff0' if v > 2 else 'fff'),
        'alt_ac': lambda v: 'background-color:rgba(0, 255, 0, %.3f)' % (min(1, v/100)),
        'filter_pass': lambda v: 'background-color: %s' % ('white' if v else 'red'),
        'RA': veff_intron_style,
        'RB': veff_intron_style,
        'RC': veff_intron_style,
        'C1N2': veff_intron_style,
        'C3N2': veff_intron_style,
        'C5N2': veff_intron_style,
        'C7N2': veff_intron_style,
        'C8N2': veff_intron_style,
        'C10N2': veff_intron_style,
        'C11N2': veff_intron_style,
        'C1N9': veff_intron_style,
        'C8N9': veff_intron_style,
        'C1N9ck': veff_intron_style,
    }                      
))

0|chrom,1|pos,2|num_alleles,3|ref,4|alt,5|alt_idx,6|alt_ac,7|filter_pass,8|annotation,9|hgvs_p,10|RA,11|RB,12|RC,13|C1N2,14|C3N2,15|C5N2,16|C7N2,17|C8N2,18|C10N2,19|C11N2,20|C1N9,21|C8N9,22|C1N9ck
2L,2362002,2,A,T,0,2,True,splice_region_variant&intron_varia,.,"('SPLICE_REGION', 'AGAP004707-PB', 3697, 'AGAP004707-PB', 4)","('SPLICE_REGION', 'AGAP004707-PB', 3697, 'AGAP004707-PB', 4)","('SPLICE_REGION', 'AGAP004707-PC', 3697, 'AGAP004707-PC', 4)",,,,,,,,,,
2L,2362003,2,C,T,0,2,True,splice_region_variant&intron_varia,.,"('SPLICE_REGION', 'AGAP004707-PB', 3698, 'AGAP004707-PB', 3)","('SPLICE_REGION', 'AGAP004707-PB', 3698, 'AGAP004707-PB', 3)","('SPLICE_REGION', 'AGAP004707-PC', 3698, 'AGAP004707-PC', 3)",,,,,,,,,,
2L,2382263,2,A,G,0,166,True,splice_region_variant&intron_varia,.,"('SPLICE_REGION', 'AGAP004707-PB', 992, 'AGAP004707-PB', 6)","('SPLICE_REGION', 'AGAP004707-PB', 992, 'AGAP004707-PB', 6)","('SPLICE_REGION', 'AGAP004707-PC', 992, 'AGAP004707-PC', 6)","('SPLICE_REGION', '4', 992, '5', 6)","('INTRONIC', '4', 992, '6', 3430)","('SPLICE_REGION', '4', 992, '5', 6)","('SPLICE_REGION', '4', 992, '5', 6)","('SPLICE_REGION', '4', 992, '5', 6)","('SPLICE_REGION', '4', 992, '5', 6)","('SPLICE_REGION', '4', 992, '5', 6)","('SPLICE_REGION', '4', 992, '5', 6)","('SPLICE_REGION', '4', 992, '5', 6)","('SPLICE_REGION', '4', 992, '5', 6)"
2L,2390126,2,C,T,0,2,True,splice_region_variant&intron_varia,.,"('SPLICE_REGION', 'AGAP004707-PB', 4340, 'AGAP004707-PB', 2)","('SPLICE_REGION', 'AGAP004707-PB', 4340, 'AGAP004707-PB', 2)","('SPLICE_REGION', 'AGAP004707-PC', 4340, 'AGAP004707-PC', 2)","('SPLICE_REGION', '6', 4340, '7', 2)","('SPLICE_REGION', '6', 4340, '7', 2)","('SPLICE_REGION', '6', 4340, '7', 2)","('SPLICE_REGION', '6', 4340, '7', 2)","('SPLICE_REGION', '6', 4340, '7', 2)","('SPLICE_REGION', '6', 4340, '7', 2)","('SPLICE_REGION', '6', 4340, '7', 2)","('SPLICE_REGION', '6', 4340, '7', 2)","('SPLICE_REGION', '6', 4340, '7', 2)","('SPLICE_REGION', '6', 4340, '7', 2)"
2L,2390130,2,C,T,0,40,True,splice_region_variant&synonymous_v,p.Gly238Gly,,,,,,,,,,,,,
2L,2390594,2,A,G,0,3,True,splice_region_variant&synonymous_v,p.Gly329Gly,,,,,,,,,,,,,
2L,2400176,2,A,G,0,1,True,splice_region_variant&intron_varia,.,"('SPLICE_REGION', 'AGAP004707-PB', 2, 'AGAP004707-PB', 2270)","('SPLICE_REGION', 'AGAP004707-PB', 2, 'AGAP004707-PB', 2270)","('SPLICE_REGION', 'AGAP004707-PC', 2, 'AGAP004707-PC', 2270)","('SPLICE_REGION', '11i+', 2, '12', 1372)","('SPLICE_REGION', '11i+', 2, '14', 2909)","('SPLICE_REGION', '11i+', 2, '14', 2909)","('SPLICE_REGION', '11i+', 2, '14', 2909)","('SPLICE_REGION', '11i+', 2, '14', 2909)","('SPLICE_REGION', '11i+', 2, '14', 2909)","('SPLICE_REGION', '11i+', 2, '14', 2909)","('SPLICE_REGION', '11i+', 2, '12', 1372)","('SPLICE_REGION', '11i+', 2, '14', 2909)","('SPLICE_REGION', '11i+', 2, '12', 1372)"
2L,2402508,2,A,T,0,1,True,missense_variant&splice_region_var,p.Gln545Leu,,,,,"('INTRONIC', '11i+', 2334, '14', 577)","('INTRONIC', '11i+', 2334, '14', 577)","('INTRONIC', '11i+', 2334, '14', 577)","('INTRONIC', '11i+', 2334, '14', 577)","('INTRONIC', '11i+', 2334, '14', 577)","('INTRONIC', '11i+', 2334, '14', 577)",,"('INTRONIC', '11i+', 2334, '14', 577)",
2L,2407888,2,T,C,0,4,True,splice_region_variant&intron_varia,.,"('SPLICE_REGION', 'AGAP004707-PB', 69, 'AGAP004707-PB', 5)","('SPLICE_REGION', 'AGAP004707-PB', 69, 'AGAP004707-PB', 5)","('SPLICE_REGION', 'AGAP004707-PC', 69, 'AGAP004707-PC', 5)","('SPLICE_REGION', '15', 69, '16', 5)","('SPLICE_REGION', '15', 69, '16', 5)","('SPLICE_REGION', '15', 69, '16', 5)","('SPLICE_REGION', '15', 69, '16', 5)","('SPLICE_REGION', '15', 69, '16', 5)","('SPLICE_REGION', '15', 69, '16', 5)","('SPLICE_REGION', '15', 69, '16', 5)","('SPLICE_REGION', '15', 69, '16', 5)","('SPLICE_REGION', '15', 69, '16', 5)","('SPLICE_REGION', '15', 69, '16', 5)"
2L,2408063,2,T,A,0,1,True,splice_region_variant&intron_varia,.,"('INTRONIC', 'AGAP004707-PB', 69, 'AGAP004707-PB', 7)","('INTRONIC', 'AGAP004707-PB', 69, 'AGA